Loading all necessary libraries and setting up the GPU config

In [1]:
import os
os.environ["CUDA_VISIBLE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle

In [2]:
gpu_options = tf.GPUOptions(allow_growth = True)
config=tf.ConfigProto(gpu_options=gpu_options)
config.gpu_options.per_process_gpu_memory_fraction = 0.34

# Question 2

In [3]:
import librosa

Setting the number of files in the training set. The max_length parameter below is used to capture the maximum length of time domain across all the train, test and validation signals. I use this parameter to pad all my signals with zeros up to the max_length. Sampling rate is set to 16000 which is common to all signals.

In [4]:
#No. of files

n_file = 1200

max_length = 200

sr = 16000

The function below will load files from a particular directory (training, test or validation). It then computes STFT for all those loaded signals, gets the absolute values, and the lengths of all the signals. Each of these parameters are also appended to separate lists and returned. Note that I also perform padding of the abs(STFT(s)) here. For example, if the STFT of a signal results in a matrix sized (513 X 90), then I pad 110 columns after the 90th column with zeroes to make the number of columns equal to 200. It turned out that I did not require to do so in the end, but you will see I have made sure that this padding does not affect the functioning and the correctness of the code, as per my knowledge.

In [16]:
#Loading the file, performing STFT, taking absolute, padding zeros as required

def loadfile(path, str_tr, flag = 0):
    list_tr = []
    list_stft = []
    list_stft_abs = []
    list_length = []
    z = ['000', '00', '0', '']
    
    for i in range(n_file):
        if (i == 0):
            j = 0
        else:
            j = int(math.log10(i))
        s, sr = librosa.load(path + str_tr + z[j] + str(i) + '.wav', sr = None)
        if (flag == 1):
            list_tr.append(s)
        
        #Calculating STFT
        stft = librosa.stft(s, n_fft= 1024, hop_length= 512)
        
        stft_len = stft.shape[1]
        
        #Appending STFT to list
        if (flag == 1):
            list_stft.append(stft)
        
        #Calculating Absolute of STFT
        stft_abs = np.abs(stft)
        
        #Padding zeros to make length 300
        stft_abs = np.pad(stft_abs, ((0,0),(0, max_length-stft_len)), 'constant')
        
        #Appending abs to list
        list_stft_abs.append(stft_abs)
        
        #Appending time-length of STFT to list
        list_length.append(stft_len)
        
    return list_tr, list_stft, list_stft_abs, list_length

Setting the path to load the training signals

In [18]:
#Path of the training signals
path = "/opt/e533/timit-homework/tr/"

Loading all the training signals below. trx, trs and trn are lists that correspond to the speech signals in time domain. X, S, and N correspond to the STFT of the signals. X_abs, S_abs, N_abs are lists that correspond to the absolute valued STFTs. X_len, S_len, and N_len correspond to the lists storing lengths of each of the loaded signals. This notation remains pretty consistent throughout the code.

In [19]:
#Loading all training noisy speech signals

trx, X, X_abs, X_len = loadfile(path, 'trx')

#Loading all training clean speech signals

trs, S, S_abs, S_len = loadfile(path, 'trs')

#Loading all training noise signals

trn, N, N_abs, N_len = loadfile(path, 'trn')

The function below computes the IBM, given the absolute valued STFTs of the clean speech and the corresponding noise signals, and returns a list of the IBMs.

In [7]:
def IBM(S, N):
    M = []
    
    for i in range(len(S)):
        m_ibm = 1 * (S[i] > N[i])
        M.append(m_ibm)
    
    return M

Fetching IBMs for the training Speech and Noise signals. These serve as the ground truth for the training of the network.

In [8]:
#Getting Binary Masks from S_abs and N_abs
M = IBM(S_abs, N_abs)

Defining the batch size and keep_pr is a placeholder for the keep probability of the dropouts.

In [9]:
batch_size = 10

#Keep probability for dropouts
keep_pr = tf.placeholder(tf.float32, ())

Defining frame size (frame_size) which is the length of the 3rd dimension of the input. num_hidden is the number of hidden layers in the LSTM cell. seq_len takes in the length of time domain for all the signals that would be passed in one batch. This was done to support the padding we had done earlier.

In [10]:
#Placeholders for input to the network

frame_size = 513
num_hidden = 256
seq_len = tf.placeholder(tf.int32, None)

q2_x = tf.placeholder(tf.float32, [None, max_length, frame_size])
q2_y = tf.placeholder(tf.float32, [None, max_length, frame_size])


Defining the RNN below. I have used one RNN with LSTM Cell with 128 hidden layers, whose weights are initialized with Xavier. Then I add a dense layer to the output so that I can get the dimensions of the network output equal to the ground truth output. I use sigmoid activation in the end to contrain output between 0 to 1. 

Note the 'dim' variable. It stores the length of time domain of the inputs in the training batch. Just storing the time domain length of the first input because the way I pass inputs in a batch, all of them have the same length in the time domain.

In [11]:
#Defining the RNN
output, state = tf.nn.dynamic_rnn(tf.nn.rnn_cell.DropoutWrapper(tf.contrib.rnn.LSTMCell(num_hidden, 
                                                         initializer = tf.contrib.layers.xavier_initializer()),
                                                                output_keep_prob = keep_pr), 
                                  q2_x, dtype=tf.float32, sequence_length=seq_len)

rnn_out = tf.layers.dense(output, 513, kernel_initializer= tf.contrib.layers.xavier_initializer())

dim = seq_len[0]

fin_out = tf.sigmoid(rnn_out)

Defining learning rate below

In [12]:
lr = 0.001

I am using MSE as the loss function. Note that I exclude the padding while calculating the MSE. This takes care that I do not underestimate the MSE.

In [13]:
cost = tf.reduce_mean(tf.losses.mean_squared_error(fin_out[:, :dim,:], q2_y[:, :dim, :]))

optimizer = tf.train.AdamOptimizer(learning_rate= lr).minimize(cost)

Intitializing the session and a saver

In [14]:
#Init all TF vars and run the session

sess = tf.Session(config = config)

saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())

I do 100 epochs

In [15]:
epochs = 100
error = np.zeros(epochs)

Training the network below. Note that the batches are consistent in each epoch (have the same set of signals), but I do change the ordering of the batches passed in each epoch. The swapaxes code swaps the axis on the signals in a manner that can be passed to the LSTM.

In [16]:
#Training the network with shuffling of training batches


for epoch in range(epochs):
    random = np.arange(0, 1200, 10)
    np.random.shuffle(random)
    for i in range(len(random)):
        start = int(random[i])
        end = int(start + batch_size)
        epoch_y = np.array(M[start:end]).swapaxes(1,2)
        epoch_x = np.array(X_abs[start:end]).swapaxes(1,2)
        seqlen = np.array(X_len[start:end])
        l, _ = sess.run([cost, optimizer], feed_dict = {q2_x: epoch_x, q2_y: epoch_y, seq_len: seqlen, keep_pr: 1})
        error[epoch] += l
    
    print('Epoch', epoch+1, 'completed out of ', epochs,'; loss: ', error[epoch])

Epoch 1 completed out of  100 ; loss:  25.92656147480011
Epoch 2 completed out of  100 ; loss:  22.47987288236618
Epoch 3 completed out of  100 ; loss:  20.69874981045723
Epoch 4 completed out of  100 ; loss:  19.52434667944908
Epoch 5 completed out of  100 ; loss:  18.64920210838318
Epoch 6 completed out of  100 ; loss:  17.93918462842703
Epoch 7 completed out of  100 ; loss:  17.531486600637436
Epoch 8 completed out of  100 ; loss:  17.18608297407627
Epoch 9 completed out of  100 ; loss:  16.84702058136463
Epoch 10 completed out of  100 ; loss:  16.597764022648335
Epoch 11 completed out of  100 ; loss:  16.41173230111599
Epoch 12 completed out of  100 ; loss:  16.02383080124855
Epoch 13 completed out of  100 ; loss:  15.841072976589203
Epoch 14 completed out of  100 ; loss:  15.670962944626808
Epoch 15 completed out of  100 ; loss:  15.514885418117046
Epoch 16 completed out of  100 ; loss:  15.291068434715271
Epoch 17 completed out of  100 ; loss:  15.30916078388691
Epoch 18 complete

In [17]:
saver.save(sess, 'q2model/q2')

'q2model/q2'

## Validation

Loading the validation set below in the same way as done for the training set previously.

In [23]:
#Loading validation set

path = "/opt/e533/timit-homework/v/"

vx, VX, VX_abs, VX_len = loadfile(path, 'vx', flag = 1)

vs, VS, VS_abs, VS_len = loadfile(path, 'vs', flag = 1)

vn, VN, VN_abs, VN_len = loadfile(path, 'vn', flag = 1)

Fetching the binary masks for the clean speech and noise validation signals.

In [19]:
#Getting Binary Masks from S_abs and N_abs
VM = IBM(VS_abs, VN_abs)

In [20]:
#Init variable to store SNR of validation sets

SNR_Val = np.zeros(1200)

Function below calculates the SNR for each of the predicted signals in the validation set. Returns the list of SNRs

In [21]:
def calc_SNR(M_pred, X, s, i):
    M_pred = 1 * (M_pred > 0.5)
    M_pred = M_pred.T
    S_pred = M_pred * X
    s_pred = librosa.istft(S_pred, win_length = 1024, hop_length = 512)
    
    nlen = min(len(s), len(s_pred))
    SNR = 10*math.log10((np.sum(s[:nlen]**2))/(np.sum((s[:nlen] - s_pred[:nlen])**2)))
    
    return SNR

Getting prediction for all validation sets from the trained network. Note that when I call the calc_SNR function, I remove the padded columns from the predicted masks. So, this is where you can see that I ensure padding does not get into the way of the authenticity of the results, again, as per my knowledge. 

In [22]:
#Getting predictions for all validation sets
for i in range(len(VX_abs)):
    epoch_x = np.zeros((1, VX_abs[i].shape[1], VX_abs[i].shape[0]))
    epoch_y = np.zeros((1, VX_abs[i].shape[1], VX_abs[i].shape[0]))
    
    epoch_x[0,:,:] = VX_abs[i].T
    epoch_y[0,:,:] = VS_abs[i].T
    
    VM_pred, val_loss = sess.run([fin_out, cost], feed_dict = {q2_x:epoch_x, q2_y:epoch_y, seq_len : np.array([VX_len[i]]), keep_pr:1})
    
    SNR_Val[i] = calc_SNR(VM_pred[0,:VX_len[i],:], VX[i], vs[i], i)

Printing the mean value of the SNR below

In [23]:
#Calculating mean SNR

print(np.mean(SNR_Val))

10.637977559016333


## Testing

Loading the validation set below.

In [25]:
#Loading test set

path = "/opt/e533/timit-homework/te/"

n_file = 400

tex, TEX, TEX_abs, TEX_len = loadfile(path, 'tex', flag = 1)

Function below writes out predicted test signals. 'tex0000.wav' is written out as 'Test_Recover_0.wav', 'tex1923.wav' as 'Test_Recover_1923.wav' and so on.

In [26]:
def test_SNR(M_pred, X, i):
    M_pred = 1 * (M_pred > 0.5)
    M_pred = M_pred.T
    S_pred = M_pred * X
    s_pred = librosa.istft(S_pred, win_length = 1024, hop_length = 512)
    
    librosa.output.write_wav('TestC/Test_Recover_' + str(i) + '.wav', s_pred, sr)

Getting prediction for the test set and storing them. I have attached all those signals in the zip file.

In [28]:
#Getting predictions for all test sets
for i in range(len(TEX_abs)):
    epoch_x = np.zeros((1, TEX_abs[i].shape[1], TEX_abs[i].shape[0]))
    epoch_y = np.zeros((1, TEX_abs[i].shape[1], TEX_abs[i].shape[0]))
    
    epoch_x[0,:,:] = TEX_abs[i].T
    #epoch_y[0,:,:] = TEabs[i].T
    
    TEM_pred= sess.run(fin_out, feed_dict = {q2_x:epoch_x, seq_len : np.array([TEX_len[i]]), keep_pr: 1})
    
    test_SNR(TEM_pred[0,:TEX_len[i],:], TEX[i], i)

In [ ]:
sess.close()